In [ ]:
p = 0x1a0111ea397fe69a4b1ba7b6434bacd764774b84f38512bf6730d2a0f6b0f6241eabfffeb153ffffb9feffffffffaaab
Fp = GF(p)
a = Fp(0x00)
b = Fp(0x04)
E = EllipticCurve(Fp, (a, b))
G = E(0x17F1D3A73197D7942695638C4FA9AC0FC3688C4F9774B905A14E3A3F171BAC586C55E83FF97A1AEFFB3AF00ADB22C6BB, 0x08B3F481E3AAA0F1A09E30ED741D8AE4FCF5E095D5D00AF600DB18CB2C04B3EDD03CC744A2888AE40CAA232946C5E7E1)
E.set_order(0x73EDA753299D7D483339D80809A1D80553BDA402FFFE5BFEFFFFFFFF00000001 * 0x396C8C005555E1568C00AAAB0000AAAB)
FpU.<T> = Fp[]
Fp2.<u> = GF(p^2, name='u', modulus=T^2 + 1)
E2 = EllipticCurve([Fp2(0), Fp2([4,4])])

E2_cofactor = 305502333931268344200999753193121504214466019254188142667664032982267604182971884026507427359259977847832272839041616661285803823378372096355777062779109

# 1. 检查是否在安全循环子群中
st10 = E(0x0F99F411A5F6C484EC5CAD7B9F9C0F01A3D2BB73759BB95567F1FE4910331D32B95ED87E36681230273C9A6677BE3A69, 0x12978C5E13A226B039CE22A0F4961D329747F0B78350988DAB4C1263455C826418A667CA97AC55576228FC7AA77D33E5)
st11 = E(0x16C2385B2093CC3EDBC0F2257E8F23E98E775F8F6628767E5F4FC0E495285B95B1505F487102FE083E65DC8E9E3A9181, 0x0F4B73F63C6FD1F924EAE2982426FC94FBD03FCEE12D9FB01BAF52BE1246A14C53C152D64ED312494A2BC32C4A3E7F9A)

st20 = E2(0x1173F10AD9F2DBEE8B6C0BB2624B05D72EEC87925F5C3633E2C000E699A580B842D3F35AF1BE77517C86AEBCA1130AE4+0x0434043A97DA28EF7100AE559167FC613F057B85451476ABABB27CFF0238A32831A0B4D14BA83C4F97247C8AC339841F*u, 0x0BEBEC70446CB91BB3D4DC5C8412915E99D612D8807C950AB06BC41583F528FDA9F42EC0FE7CD2991638187EF44258D3+0x19528E3B5C90C73A7092BB9AFDC73F86C838F551CCD9DBBA5CC6244CF76AB3372193DBE5B62383FAAE728728D4C1E649*u)

st21 = E2(0x165830F15309C878BFE6DD55697860B8823C1AFBDADCC2EF3CD52B56D4956C05A099D52FE4545816830C525F5484A5FA + 0x179E34EB67D9D2DD32B224CDBA57D4BB7CF562B4A3E33382E88F33882D91663B14738B6772BF53A24653CE1DD2BFE2FA * u, 0x150598FC4225B44437EC604204BE06A2040FD295A28230B789214B1B12BF9C9DAE6F3759447FD195E92E2B42E03B5006 + 0x12E23B19E117418C568D4FF05B7824E5B54673C3C08D8BCD6D8D107955287A2B075100A51C81EBA44BF5A1ABAD4764A8 * u)


# expected: (0: 1: 0)
print(st10 * 0x73EDA753299D7D483339D80809A1D80553BDA402FFFE5BFEFFFFFFFF00000001)

# 2. 打印实际 order并 factor
print("Order of g1: ", st10.order())
print("Factors of order of g1:", factor(g1_order))
# twist curve order same as base curve order

# 3. 曲线 cofactor 
print("Cofactor of base curve factors: " + str(factor(0x396C8C005555E1568C00AAAB0000AAAB)))
print("Cofactor of twist curve factors: " + str(factor(E2_cofactor)))

# 4. 计算 s mod g1_order, s mod g2_order
s_n1 = discrete_log(st11 * 52435875175126190479447740508185965837690552500527637822603658699938581184513, st10 * 52435875175126190479447740508185965837690552500527637822603658699938581184513, operation = '+')

s_n2 = discrete_log(st21 * 402096035359507321594726366720466575392706800671181159425656785868777272553337714697862511267018014931937703598282857976535744623203249 * 52435875175126190479447740508185965837690552500527637822603658699938581184513 * 13 * 23, st20 * 402096035359507321594726366720466575392706800671181159425656785868777272553337714697862511267018014931937703598282857976535744623203249 * 52435875175126190479447740508185965837690552500527637822603658699938581184513 * 13 * 23, 13 * 23 * 2713 * 11953 * 262069, operation = '+')

print(s_n1)
print(s_n2)

print("Test: ", st20 * 402096035359507321594726366720466575392706800671181159425656785868777272553337714697862511267018014931937703598282857976535744623203249 * 52435875175126190479447740508185965837690552500527637822603658699938581184513 * 13 * 23 *2713 * 11953 * 262069 )


# 4. CRT
s_tag = crt([s_n1, s_n2], [3 * 11 * 10177 * 859267 * 52437899, 2713 * 11953 * 262069])
print("s_tag: ", s_tag)
print(len(s_tag.bits()))

n = 3 * 11 * 10177 * 859267 * 52437899 * 2713 * 11953 * 262069
k = 893753
while True:
    s = n * k + s_tag
    k += 1
    if st10 * s == st11:
        print(s)
        break

(2622753952425151039330313939300837055090408924462582151152718087618982514538255115733407723427284137667587248467726 : 1008846640337108832223130256947579587620502671179283038532135212440984764733818392245359127024407649702099419925203 : 1)
Order of g1:  793479390729215512516507951283169066088130679960393952059283337873017453583023682367384822284289
Factors of order of g1: 3 * 11 * 10177 * 859267 * 52437899 * 52435875175126190479447740508185965837690552500527637822603658699938581184513
Cofactor of base curve factors: 3 * 11^2 * 10177^2 * 859267^2 * 52437899^2
Cofactor of twist curve factors: 13^2 * 23^2 * 2713 * 11953 * 262069 * 402096035359507321594726366720466575392706800671181159425656785868777272553337714697862511267018014931937703598282857976535744623203249
2335387132884273659
6942769366567
Test:  (0 : 1 : 0)
s_tag:  62308043734996521086909071585406
106
114939083266787167213538091034071020048


(2622753952425151039330313939300837055090408924462582151152718087618982514538255115733407723427284137667587248467726 : 1008846640337108832223130256947579587620502671179283038532135212440984764733818392245359127024407649702099419925203 : 1)
Order of g1:  793479390729215512516507951283169066088130679960393952059283337873017453583023682367384822284289
Factors of order of g1: 3 * 11 * 10177 * 859267 * 52437899 * 52435875175126190479447740508185965837690552500527637822603658699938581184513
Cofactor of base curve factors: 3 * 11^2 * 10177^2 * 859267^2 * 52437899^2
Cofactor of twist curve factors: 13^2 * 23^2 * 2713 * 11953 * 262069 * 402096035359507321594726366720466575392706800671181159425656785868777272553337714697862511267018014931937703598282857976535744623203249
2335387132884273659
6942769366567
Test:  (0 : 1 : 0)
s_tag:  62308043734996521086909071585406
106
114939083266787167213538091034071020048
